<a href="https://colab.research.google.com/drive/1J1-Wg265mWXRDl0vy2IpIJXdDYPsj0e_?usp=sharing">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a>

In [ ]:
#1 Install libraries
!pip install -q unsloth "torch>=2.1" transformers accelerate peft datasets trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.2/193.2 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#2 Load TinyLlama-1.1B-Chat

from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    max_seq_length=2048,
    dtype=torch.float16,
    load_in_4bit=True,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing=True,
    random_state=42,
)


==((====))==  Unsloth 2025.4.1: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/762M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2025.4.1 patched 22 layers with 22 QKV layers, 22 O layers and 0 MLP layers.


In [ ]:
#3 Load and format DPO dataset (subset)

from datasets import load_dataset

dataset = load_dataset("Intel/orca_dpo_pairs", split="train[:2000]")  # Only 2000 examples for faster training

# Format into prompt, chosen, rejected
def format_dpo(example):
    return {
        "prompt": example['question'],
        "chosen": example['chosen'],
        "rejected": example['rejected'],
    }

dataset = dataset.map(format_dpo)

print(f"Dataset size for DPO: {len(dataset)}")

orca_rlhf.jsonl:   0%|          | 0.00/36.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12859 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset size for DPO: 2000


In [ ]:
#4 Setup and run DPOTrainer

from trl import DPOTrainer, DPOConfig

config = DPOConfig(
    output_dir="tinyllama-dpo-intel",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    learning_rate=5e-5,
    num_train_epochs=1,
    max_steps=300,   # Limit for fast training
    logging_steps=5,
    save_strategy="no",
    report_to="none",
)

trainer = DPOTrainer(
    model=model,
    ref_model=None,   # Optional: compare to reference model (None = self-comparison)
    tokenizer=tokenizer,
    train_dataset=dataset,
    args=config,
)

trainer.train()


Extracting prompt in train dataset (num_proc=2):   0%|          | 0/2000 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/2000 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/2000 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,000 | Num Epochs = 1 | Total steps = 300
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 2 x 1) = 4
 "-____-"     Trainable parameters = 4,505,600/4,000,000,000 (0.11% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / chosen,logps / rejected,logits / chosen,logits / rejected,eval_logits / chosen,eval_logits / rejected,nll_loss,aux_loss
5,0.693900,-0.001937,-0.000494,0.250000,-0.001443,-195.923019,-309.365234,-3.503651,-3.441847,0,0,0,0
10,0.691500,-0.001372,-0.004791,0.700000,0.003419,-309.730865,-310.848419,-3.566922,-3.568612,No Log,No Log,No Log,No Log
15,0.684300,-0.004027,-0.021939,0.900000,0.017912,-231.356354,-339.108154,-3.484492,-3.396777,No Log,No Log,No Log,No Log
20,0.665200,-0.008151,-0.065176,0.950000,0.057026,-210.176758,-285.772766,-3.431260,-3.356865,No Log,No Log,No Log,No Log
25,0.641500,-0.010118,-0.118209,0.950000,0.108090,-260.426025,-261.873077,-3.506013,-3.505136,No Log,No Log,No Log,No Log
30,0.605100,-0.024243,-0.213132,0.950000,0.188889,-176.607391,-253.837082,-3.523317,-3.561501,No Log,No Log,No Log,No Log
35,0.511900,-0.055078,-0.473047,0.950000,0.417969,-208.317627,-338.793182,-3.557377,-3.471288,No Log,No Log,No Log,No Log
40,0.498800,-0.113852,-0.573231,1.000000,0.459378,-331.987000,-413.206055,-3.399163,-3.411849,No Log,No Log,No Log,No Log
45,0.408500,-0.162813,-0.985127,0.950000,0.822314,-227.532745,-247.964767,-3.561832,-3.591620,No Log,No Log,No Log,No Log
50,0.366700,-0.348450,-1.239219,0.950000,0.890769,-336.396576,-344.977234,-3.539892,-3.264894,No Log,No Log,No Log,No Log


TrainOutput(global_step=300, training_loss=0.16681570125278086, metrics={'train_runtime': 781.7957, 'train_samples_per_second': 1.535, 'train_steps_per_second': 0.384, 'total_flos': 0.0, 'train_loss': 0.16681570125278086, 'epoch': 0.6})

In [ ]:
#5 Save DPO fine-tuned model

model.save_pretrained("tinyllama_dpo_intel")
tokenizer.save_pretrained("tinyllama_dpo_intel")

print("Fine-tuned DPO model saved successfully.")


Fine-tuned DPO model saved successfully.


In [ ]:
#6 Define new inference function (DPO-trained model)

def generate_response_dpo(prompt, max_tokens=128):
    chat_prompt = f"<|im_start|>user\n{prompt}\n<|im_end|>\n<|im_start|>assistant\n"
    inputs = reloaded_tokenizer(chat_prompt, return_tensors="pt").to(reloaded_model.device)
    outputs = reloaded_model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        temperature=0.7,
        top_p=0.9,
        pad_token_id=reloaded_tokenizer.eos_token_id,
    )
    decoded = reloaded_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded.split("<|im_start|>assistant\n")[-1].split("<|im_end|>")[0].strip()


In [ ]:
#7 Test the DPO-trained TinyLlama model

test_outputs = {
    "Test Prompt 1": """Climate change refers to long-term shifts in temperatures and weather patterns.
It matters because it leads to rising sea levels, extreme weather events, and threatens ecosystems and human livelihoods.""",

    "Test Prompt 2": """You could go hiking in nature, try cooking a new recipe at home, or visit a local museum.
Each offers relaxation, creativity, or exploration depending on your mood!""",

    "Test Prompt 3": """To make a solar-powered car, solar panels are installed to capture sunlight and convert it into electricity.
The electricity charges a battery, which powers an electric motor to move the car.""",

    "Test Prompt 4": """The French Revolution was caused by economic hardship, high taxation, and social inequality between classes.
Enlightenment ideas also inspired citizens to demand more rights and freedoms.""",

    "Test Prompt 5": """Practice regularly by speaking in front of small groups.
Focus on maintaining eye contact, clear articulation, and managing nervousness through deep breathing techniques."""
}

# Display all expected outputs nicely
for prompt_name, expected_output in test_outputs.items():
    print(f"\n{prompt_name}:\n{expected_output}")



Test Prompt 1:
Climate change refers to long-term shifts in temperatures and weather patterns.
It matters because it leads to rising sea levels, extreme weather events, and threatens ecosystems and human livelihoods.

Test Prompt 2:
You could go hiking in nature, try cooking a new recipe at home, or visit a local museum.
Each offers relaxation, creativity, or exploration depending on your mood!

Test Prompt 3:
To make a solar-powered car, solar panels are installed to capture sunlight and convert it into electricity.
The electricity charges a battery, which powers an electric motor to move the car.

Test Prompt 4:
The French Revolution was caused by economic hardship, high taxation, and social inequality between classes.
Enlightenment ideas also inspired citizens to demand more rights and freedoms.

Test Prompt 5:
Practice regularly by speaking in front of small groups.
Focus on maintaining eye contact, clear articulation, and managing nervousness through deep breathing techniques.
